In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.3.0-bin-hadoop3')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_part_2').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/09 06:37:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [6]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [9]:
for item in data.head(2)[1]:
    print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [17]:
data.columns


['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [5]:
assembler = VectorAssembler(inputCols=[ 'Avg Session Length',
                                         'Time on App',
                                         'Time on Website',
                                         'Length of Membership',
                                         'Yearly Amount Spent'], 
                                           outputCol='features')

In [6]:
output = assembler.transform(data)

In [21]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [22]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [23]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [7]:
final_data = output.select('features', 'Yearly Amount Spent')

In [26]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [8]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [9]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                346|
|   mean|  495.4691217903679|
| stddev|  79.65430175919813|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [10]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [11]:
lr_model = lr.fit(train_data)

22/09/09 06:38:10 WARN Instrumentation: [c4d6e4ee] regParam is zero, which might cause numerical instability and overfitting.
22/09/09 06:38:10 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/09/09 06:38:10 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/09/09 06:38:11 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [12]:
test_results = lr_model.evaluate(test_data)

In [18]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-6.82121026329696...|
|5.400124791776761...|
|-4.32009983342140...|
|-7.27595761418342...|
|-2.78532752417959...|
|2.103206497849896...|
|1.324451659456826...|
|9.094947017729282...|
|-4.37694325228221...|
|5.400124791776761...|
|-7.10542735760100...|
|-3.80850906367413...|
|-1.11981535155791...|
|9.72022462519817E-12|
|-8.12860889709554...|
|3.410605131648481...|
|1.602984411874786...|
|3.410605131648481...|
|5.115907697472721...|
|4.718003765447065...|
+--------------------+
only showing top 20 rows



In [16]:
test_results.rootMeanSquaredError

1.8309843834555353e-11

In [14]:
test_results.r2

1.0

In [15]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [17]:
unlabeled_data = test_data.select('features')

In [18]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.7377203726281...|
|[30.9716756438877...|
|[31.1695067987115...|
|[31.2834474760581...|
|[31.3662121671876...|
|[31.4252268808548...|
|[31.4474464941278...|
|[31.5171218025062...|
|[31.5257524169682...|
|[31.6253601348306...|
|[31.6610498227460...|
|[31.6739155032749...|
|[31.7216523605090...|
|[31.8124825597242...|
|[31.8164283341993...|
|[31.8530748017465...|
|[31.8627411090001...|
|[31.8854062999117...|
|[31.9048571310136...|
|[31.9262720263601...|
+--------------------+
only showing top 20 rows



In [19]:
predictions = lr_model.transform(unlabeled_data)

In [20]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.7377203726281...|461.78074219625336|
|[30.9716756438877...| 494.6386097569106|
|[31.1695067987115...|427.35653080231185|
|[31.2834474760581...| 591.7810894257099|
|[31.3662121671876...|430.58888255649873|
|[31.4252268808548...| 530.7667186547595|
|[31.4474464941278...| 418.6027420952217|
|[31.5171218025062...|275.91842065038736|
|[31.5257524169682...|443.96562680987876|
|[31.6253601348306...| 376.3369007569179|
|[31.6610498227460...|416.35835357990373|
|[31.6739155032749...|475.72506790983994|
|[31.7216523605090...|347.77692663187264|
|[31.8124825597242...|392.81034498379205|
|[31.8164283341993...|501.12249150362555|
|[31.8530748017465...| 459.2851234623524|
|[31.8627411090001...| 556.2981411740449|
|[31.8854062999117...| 390.1032729724652|
|[31.9048571310136...|473.94985742279005|
|[31.9262720263601...|392.20493344435226|
+--------------------+------------